##Loading text data

In [ ]:
path = '/content/drive/MyDrive/'
file = open(path + 'fashion_corpus_clean.txt', 'r')
text = file.read()
file.close()

In [ ]:
tokens = text[:500000].split() #part of the whole dataset
tokens = [word for word in tokens if word.isalpha()]

print('Number of tokens: ', len(tokens))
print('Number of unique tokens: ', len(set(tokens)))

Number of tokens:  57075
Number of unique tokens:  14261


##Creating sequences

In [ ]:
length = 20 + 1 #length of sequences

sequences = list()
for i in range(length, len(tokens)):
	seq = tokens[i-length:i]
	line = ' '.join(seq)
	sequences.append(line)

print('number of sequences: ', len(sequences))

out_filename = 'fashion_sequences.txt'


number of sequences:  57054


In [ ]:
#save sequences in text file
data = '\n'.join(sequences)
file = open(path+out_filename, 'w')
file.write(data)
file.close()

##Creating model

In [ ]:
from numpy import array
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout

In [ ]:
path = '/content/drive/MyDrive/'
file = open(path + 'fashion_sequences.txt', 'r')
doc = file.read()
file.close()
lines = doc.split('\n')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

The first layer is an **Embedding layer** which stores one vector per word. It converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training, words with similar meanings often have similar vectors.

**LSTM layer** (Long Short-Term Memory). Enables the model to learn what information to store in long term memory and what to get rid of.


**Dropout layer** prevents model from overfitting to training data.

**Flatten layer** reshapes tensor to have the shape that is equal to the number of elements contained in tensor


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           1426100   
_________________________________________________________________
lstm (LSTM)                  (None, 20, 100)           80400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            30200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 32)            10624     
_________________________________________________________________
flatten (Flatten)            (None, 640)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                10256     
_________________________________________________________________
dropout (Dropout)            (None, 16)                0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=64, epochs=200)

Epoch 1/200
892/892 [==============================] - 85s 89ms/step - loss: 8.3292 - accuracy: 0.0407
Epoch 2/200
892/892 [==============================] - 79s 88ms/step - loss: 8.0168 - accuracy: 0.0419
Epoch 3/200
892/892 [==============================] - 78s 88ms/step - loss: 7.9303 - accuracy: 0.0425
Epoch 4/200
892/892 [==============================] - 78s 87ms/step - loss: 7.8403 - accuracy: 0.0427
Epoch 5/200
892/892 [==============================] - 78s 87ms/step - loss: 7.7397 - accuracy: 0.0429
Epoch 6/200
892/892 [==============================] - 78s 87ms/step - loss: 7.6572 - accuracy: 0.0434
Epoch 7/200
892/892 [==============================] - 77s 87ms/step - loss: 7.5790 - accuracy: 0.0431
Epoch 8/200
892/892 [==============================] - 78s 88ms/step - loss: 7.5016 - accuracy: 0.0437
Epoch 9/200
892/892 [==============================] - 78s 87ms/step - loss: 7.4146 - accuracy: 0.0435
Epoch 10/200
892/892 [==============================] - 78s 88ms/step - l

In [ ]:
model.save(path+'model_3.h5')
dump(tokenizer, open(path+'tokenizer.pkl', 'wb'))

In [ ]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

def generate(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        pred = model.predict(encoded, verbose=0)
        yhat = np.argmax(pred,axis=1)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

model = load_model(path+'model_3.h5')

tokenizer = load(open(path+'tokenizer.pkl', 'rb'))

init_text = "Sukienka "
print(init_text + '\n')

generated = generate(model, tokenizer, seq_length, init_text, 15)
print(generated)

Sukienka 

artystki kosmetyczki czy artystki kosmetyczki czy zabrała ulubieńców artystki różnorodnych wynik wynik wynik artystki artystki
